In [3]:
!pip install --upgrade --quiet langchain-openai tiktoken chromadb langchain


In [4]:
!pip install git

ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


In [5]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [6]:
# Clone
repo_path = "/Users/praveenreddy/GenAI/GenAI_Practice/LangChain_docs/test_repo"
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [7]:
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf8-encoding.py"],
    parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [8]:
documents = loader.load()

In [9]:
len(documents)

308

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)

In [11]:
len(texts)

941

In [12]:
texts[0]

Document(page_content='"""**sys_info** prints information about the system and langchain packages\nfor debugging purposes.\n"""\n\nfrom typing import Sequence', metadata={'source': '/Users/praveenreddy/GenAI/GenAI_Practice/LangChain_docs/test_repo/libs/core/langchain_core/sys_info.py', 'language': <Language.PYTHON: 'python'>})

In [15]:
OPENAI_API_KEY = ""

In [16]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [17]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model="gpt-4")

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

In [19]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)


In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

In [22]:
qa = create_retrieval_chain(retriever_chain, document_chain)

In [23]:
question = "What is a RunnableBinding?"
result = qa.invoke({"input": question})
result["answer"]

'A RunnableBinding is a class in LangChain that allows arguments to be bound to a Runnable, returning a new Runnable. This is especially useful when a runnable in a chain requires an argument that is not in the output of the previous runnable or included in the user input. The bound arguments are provided in the form of keyword arguments (kwargs). An example usage of RunnableBinding is provided in the context, where it is used to bind a stop parameter to a ChatOllama model in a chain.'

In [26]:
print(result) #.pretty_print()

{'input': 'What is a RunnableBinding?', 'context': [Document(page_content='# from langchain_core.runnables.base import RunnableBinding\n\n\n# class RunnableLearnable(RunnableBinding):\n#     def __init__(self, *args, **kwargs):\n#         super().__init__(*args, **kwargs)\n#         self.parameters = []\n\n#     def backward(self):\n#         for param in self.parameters:\n#             param.backward()\n\n#     def update(self, optimizer):\n#         for param in self.parameters:\n#             optimizer.update(param)', metadata={'language': 'python', 'source': '/Users/praveenreddy/GenAI/GenAI_Practice/LangChain_docs/test_repo/libs/core/langchain_core/runnables/learnable.py'}), Document(page_content='async for chunk in input:\n            if not got_first_val:\n                final = adapt_first_streaming_chunk(chunk)  # type: ignore\n                got_first_val = True\n            else:\n                # Make a best effort to gather, for any type that supports `+`\n              

In [27]:
questions = [
    "What classes are derived from the Runnable class?",
    "What one improvement do you propose in code in relation to the class hierarchy for the Runnable class?",
]

for question in questions:
    result = qa.invoke({"input": question})
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What classes are derived from the Runnable class? 

**Answer**: The classes derived from the Runnable class mentioned in the given context are:

1. RunnableLambda
2. RunnableBinding
3. RunnableLearnable
4. RunnableSerializable
5. RunnableWithFallbacks (indirectly mentioned, not explicitly defined in the given context) 

-> **Question**: What one improvement do you propose in code in relation to the class hierarchy for the Runnable class? 

**Answer**: One potential improvement could be to create an abstract base class for the Runnable class to better organize the hierarchy and enforce the implementation of certain methods. This would ensure that all subclasses of Runnable would have to implement these methods, providing a consistent interface across all types of Runnables. This can be especially helpful in large codebases to ensure consistency and clarity in the design of the software. 

